In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

model = SentenceTransformer('all-MiniLM-L6-v2')

df = pd.read_csv('../data/filtered_events_country_code.csv', delimiter=',')
"""
#topics more expansive
topics = [
    "women rights", "health care", "farmers", "environment", "labor rights", "infrastructure", "palestine-israel conflict", "climate", "anti right wing", "immigration", "eviction","youth violence", "tourism",
    "unjust law enforcement", "democracy", "ukraine-russia war", "discrimination",
    "education", "housing", "culture", "social welfare", "political rights", "public services", "policital criticism", "animal welfare",
]
"""
#with descriptions
topics = [
    "pro palestine: Content supportive of Palestinian rights, sovereignty, and resistance to Israeli occupation or military actions.",
    "pro israel: Content supportive of the state of Israel, its right to exist, defend itself, and maintain security.",
    "climate: Discussions related to climate change, global warming, fossil fuels, greenwashing, and sustainability.",
    "consumerism: Criticism or commentary on materialism, overconsumption, capitalism, or commercial culture.",
    "migration: Topics about immigration, refugees, asylum seekers, border policies, and demographic movement.",
    "labor rights: Advocacy for workers' rights, fair wages, unions, pensions, salary equity, and improved working conditions.",
    "racism: Issues related to racial discrimination, systemic injustice, white supremacy, or anti-racism advocacy.",
    "higher education: Content focused on universities, student protests, tuition fees, academic freedom, or campus politics.",
    "corruption: Allegations or discussions of government misconduct, bribery, embezzlement, or political scandals.",
    "anti-government: Expressions of dissent or protest against state power, authoritarianism, police violence, or national institutions.",
    "peace: Calls for ceasefires, nonviolence, diplomacy, coexistence, or reconciliation between opposing groups.",
    "lgbtq rights: Advocacy for the rights, safety, and equality of LGBTQ individuals, including issues of discrimination, marriage, identity, and visibility.",
    "women rights: Advocacy for gender equality, women's rights, feminism, and opposition to gender-based violence or discrimination.",
    "health care: Discussions related to medical services, hospitals, public health, and access to care, including nurseries and healthcare reform.",
    "farmers: Topics involving agricultural workers, land rights, farming practices, and rural economic issues.",
    "environment: Coverage of biodiversity, pollution, forest management (e.g., planned felling), water systems, and environmental toxins such as PFAS.",
    "infrastructure: Public infrastructure including roads, public transit, railways, bike lanes, and pedestrian zones; also incidents like traffic accidents or structural collapses.",
    "eviction: Protests or discussions about housing evictions, tenants' rights, and access to affordable housing.",
    "youth violence: Reports or protests concerning violence involving minors or youth, often in schools or communities.",
    "anti crime: Efforts to reduce organized crime, enforce security measures, and end gang violence or criminal networks.",
    "tourism: Matters related to tourism, travel policy, or the effects of tourism on communities and the environment.",
    "unjust law enforcement: Criticism of police actions such as brutality, violent interventions, or criminalization of protests.",
    "democracy: Support for democratic governance, fair elections, referenda, and democratic institutions.",
    "ukraine-russia war: Content related to the Russian invasion of Ukraine, Ukrainian refugees, military conflict, and international response.",
    "discrimination: Content addressing unfair treatment based on race, gender, religion, disability, or other identity markers.",
    "education: Issues related to schools, universities, teachers, students, and academic policy or reform.",
    "housing: Concerns around residential developments, housing access, homelessness, and urban planning.",
    "culture: Content about arts, heritage, cultural institutions, and expressions of identity through culture.",
    "social welfare: Topics about government support systems such as unemployment benefits, social housing, and welfare services.",
    "public services: Issues involving access to or quality of services like public transport, utilities, and administrative support.",
    "political: Political dissent, opposition to government leaders, far-right politics, political imprisonment, or calls for resignations.",
    "animal welfare: Protection and rights of animals, prevention of cruelty, and advocacy for humane treatment.",
    "cyprus conflict: Content referencing the Cyprus conflict, including the Turkish invasion and political division of the island.",
    "military: Opposition to military spending, weapons production, or armed conflict in general.",
 
    "local policy: Issues specific to municipal or regional governance, such as local council decisions and planning.",
    "sudan conflict: Discussions of the Sudanese civil war, humanitarian crisis, and violence in Sudan."
]

"""
#topics strictly from slides
topics = [
    "pro palestine", 
    "pro israel",
    "climate",
    "consumerism",
    "migration",
    "labor rights",
    "racism",
    "higher education",
    "corruption",
    "anti-government",
    "peace"
]
"""
"""
#with descriptions
topics = [
    "pro palestine: Content supportive of Palestinian rights, sovereignty, and resistance to Israeli occupation or military actions.",
    "pro israel: Content supportive of the state of Israel, its right to exist, defend itself, and maintain security.",
    "climate: Discussions related to climate change, global warming, environmental protection, and sustainability.",
    "consumerism: Criticism or commentary on materialism, overconsumption, capitalism, or commercial culture.",
    "migration: Topics about immigration, refugees, asylum seekers, border policies, and demographic movement.",
    "labor rights: Advocacy for workers' rights, fair wages, unions, labor laws, or critiques of workplace exploitation.",
    "racism: Issues related to racial discrimination, systemic injustice, white supremacy, or anti-racism advocacy.",
    "higher education: Content focused on universities, student protests, tuition fees, academic freedom, or campus politics.",
    "corruption: Allegations or discussions of government misconduct, bribery, embezzlement, or political scandals.",
    "anti-government: Expressions of dissent or protest against state power, authoritarianism, police violence, or national institutions.",
    "peace: Calls for ceasefires, nonviolence, diplomacy, coexistence, or reconciliation between opposing groups.",
    "lgbtq rights: Advocacy for the rights, safety, and equality of LGBTQ individuals, including issues of discrimination, marriage, identity, and visibility."
]"""

topic_embeddings = model.encode(topics, convert_to_tensor=True)

def assign_topic(text):
    note_embedding = model.encode(text, convert_to_tensor=True)
    cosine_scores = util.cos_sim(note_embedding, topic_embeddings)
    best_topic_idx = cosine_scores.argmax().item()
    return topics[best_topic_idx]
 
df['predicted_topic'] = df['clean_notes'].apply(assign_topic)

In [8]:
df = df[['notes', 'clean_notes', 'predicted_topic']]
df.to_csv("../data/notes_with_inferred_topics_ESS_strictly_slides.csv", index=False)